In [1]:
import os
import csv

import CFV_Extractor as CE

In [2]:
ifWords = ["if","else","else if"]
keyWords = ["if","while","for","switch","else","else if"]

In [3]:
def getGpickleList(path):
    name_list = []
    for filename in os.listdir(path):
        if filename.endswith(".gpickle"):
            name_list.append(os.path.join(path,filename))
    return name_list

In [4]:
def scanIFinEnvironment(nodeSet):
    node_if_numbers = [0] *  len(nodeSet)
    for index in range(0,len(nodeSet)):
        cfv = nodeSet[index]["cfv"]
        for i in range(0,len(cfv)):
            if cfv[i] == "if" or cfv[i] == "else if":
                node_if_numbers[index] += 1
    return node_if_numbers

In [5]:
def scanInputDeepth(nodeSet):
    node_if_numbers = scanIFinEnvironment(nodeSet)
    high = 0
    for i in range(0,len(node_if_numbers)):
        if node_if_numbers[i]>high:
            high = node_if_numbers[i]
    return high

In [6]:
def countWeight(cfv):
        weight = 0
        cur_weight = 0
        record = []
        for item in cfv:
            if item in keyWords:
                record.append(item)
                if len([x for x in record if x in ifWords]) > weight:
                    weight = len([x for x in record if x in ifWords])
            else:
                if item == "}":
                    if len(record) > 0:
                        record.pop()          
        return weight
    
def initWeight(cfvSet):
        weight = []
        for item in cfvSet:
            weight.append(countWeight(item["cfv"]))
        return weight

In [7]:
def scanVulDeepth(nodeSet,extractor):
    cfvWeight = initWeight(nodeSet)

    allPath = []

    for target in range(1,len(cfvWeight)):
        allPath.append(list(extractor.getAllPath(0,target)))

    #print(allPath)
    #print(cfvWeight)

    high = 0
    for i in range(0,len(cfvWeight)-1):
        for path in allPath[i]:
            score = sum([cfvWeight[x] for x in path])
            if score >high:
                high = score
                
    return high

In [8]:
gpickle_path = "..\gpickle"

In [9]:
gpickle_list = getGpickleList(gpickle_path)

In [10]:
with open("gpickle.csv","w",newline="") as datacsv:
    csvwriter = csv.writer(datacsv,dialect =("excel"))
    
    header = ["file_path","input_deepth","vul_deepth"]
    
    csvwriter.writerow(header)
    
    for i in range(0,len(gpickle_list)):
        extractor = CE.CFV_Extractor(gpickle_list[i])
        nodeSet = extractor.getAllCFV()
        input_deepth = scanInputDeepth(nodeSet)
        vul_deepth = scanVulDeepth(nodeSet,extractor)
        
        line = [gpickle_list[i],str(input_deepth),str(vul_deepth)]
        csvwriter.writerow(line)
        
        del extractor
        
    datacsv.close()